In [1]:
# facilitate imports from project root
import sys

sys.path.append("..")

In [2]:
import os
from pathlib import Path

from dotenv import load_dotenv
from tqdm import tqdm

from chase_rank.wrappers import (
    StravaHandler, ActivityHandler, TrackHandler, StravaUserHandler
)

In [3]:
load_dotenv("../.env")

USER_PATH = Path(os.getenv("USER_JSON_PATH", "../data/users.json"))
ACTIVITY_PATH = Path(os.getenv("ACTIVITY_PATH", "../data/activities.parquet"))
TRACK_PATH = Path(os.getenv("TRACK_PATH", "../data/tracks"))

STRAVA_CLIENT_ID = os.getenv("STRAVA_CLIENT_ID")
STRAVA_CLIENT_SECRET = os.getenv("STRAVA_CLIENT_SECRET")

# Initialize Handlers

In [4]:
user_handler = StravaUserHandler(USER_PATH)
strava = StravaHandler(
    client_id=STRAVA_CLIENT_ID,
    client_secret=STRAVA_CLIENT_SECRET,
    user_handler=user_handler,
)

activities = ActivityHandler(ACTIVITY_PATH, strava)
tracks = TrackHandler(TRACK_PATH, strava)

In [5]:
# get all rides with a start and end point
rides = activities.activities[
    (activities.activities["sport_type"] == "Ride") &
    (~activities.activities["start_lat"].isna()) &
    (~activities.activities["end_lat"].isna())]

### Get a single track for an activity

In [6]:
activity = rides.iloc[0]

track = tracks.get(
    activity_id=int(activity.strava_id),
    user_id=int(activity.user_id),
    start_time=activity.start_date
)

track["geometry"].explore()

### Load all tracks we haven't stored yet
This will attempt to reload activities without valid geometry

In [7]:
for activity_id, user_id, start_time in tqdm(
        rides[["strava_id", "user_id", "start_date"]].values):
    track = tracks.get(
        activity_id=int(activity_id),
        user_id=int(user_id),
        start_time=start_time
    )

  1%|          | 8/731 [00:02<03:31,  3.42it/s]

No Rate Limit In Headers
No Usage In Headers
No Rate Limit In Headers
No Usage In Headers


 15%|█▍        | 108/731 [03:20<22:49,  2.20s/it]

15min Rate Limit Exceeded


 28%|██▊       | 208/731 [17:21<15:33,  1.78s/it]    

...

 83%|████████▎ | 608/731 [1:16:40<01:51,  1.10it/s]    

15min Rate Limit Exceeded


 97%|█████████▋| 708/731 [1:32:16<00:50,  2.22s/it]   

15min Rate Limit Exceeded


100%|██████████| 731/731 [1:45:23<00:00,  8.65s/it]   


In [8]:
f"{len(tracks.track_id_list)} tracks stored"

'731 tracks stored'